In [ ]:
# importando bibliotecas

# numericas
import numpy as np

# manipulacao de dados
import pandas as pd

# graficas
import matplotlib.pyplot as plt
import seaborn as sns

# utilidades
# from google.colab import drive

import warnings
warnings.filterwarnings('ignore')

from collections import Counter

Para esta estapa, foram testadas diversas tabelas para realizar uma junção entre um código CID-10 e sua descrição.

 A maioria das tabelas ou não tinhamos confiança na origem, ou não possuiam todos os códigos necessários.
Por fim, utilizamos uma tabela do datasus, que foi manipulada para atender
nossas necessidades.

Fonte das tabela: https://www.cdc.gov/nchs/icd/Comprehensive-Listing-of-ICD-10-CM-Files.htm

Fonte Datasus: https://www.gov.br/previdencia/pt-br/assuntos/previdencia-social/saude-e-seguranca-do-trabalhador/acidente_trabalho_incapacidade/tabelas-cid-10

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/ImagineDogs/TranstornosMentais/main/df_limpo.csv')

In [ ]:
# Tabela equivalente aos grupos do CID-10
cid_df = pd.read_csv('https://raw.githubusercontent.com/ImagineDogs/TranstornosMentais/main/Tabela%20CID-10.csv')
cid_df.head()

In [ ]:
# Aplicando tratamento aos dados para realizar a junção com a tabela de descrições
# Não devem haver pontos nos códigos e todos devem estar em caixa alta
df['cid10_diagnostico'] = df['cid10_diagnostico'].loc[~df['cid10_diagnostico'].isnull()].apply(lambda x: str(x).upper()[:str(x).find('.')] if str(x).find('.') != -1 else str(x).upper())
df['cid10_seg_diagnostico'] = df['cid10_seg_diagnostico'].loc[~df['cid10_seg_diagnostico'].isnull()].apply(lambda x: str(x).upper()[:str(x).find('.')] if str(x).find('.') != -1 else str(x).upper())
df['cid10_diagnostico_alta'] = df['cid10_diagnostico_alta'].loc[~df['cid10_diagnostico_alta'].isnull()].apply(lambda x: str(x).upper()[:str(x).find('.')] if str(x).find('.') != -1 else str(x).upper())
df['cid10_seg_diagnostico_alta'] = df['cid10_seg_diagnostico_alta'].loc[~df['cid10_seg_diagnostico_alta'].isnull()].apply(lambda x: str(x).upper()[:str(x).find('.')] if str(x).find('.') != -1 else str(x).upper())

In [ ]:
# Junção entre as tabelas
df = df.merge(cid_df, how='left', left_on='cid10_diagnostico', right_on='Codigo').rename(columns={'Descricao': 'descricao_diagnostico'}).drop(['Codigo'], axis=1)
df = df.merge(cid_df, how='left', left_on='cid10_seg_diagnostico', right_on='Codigo').rename(columns={'Descricao': 'descricao_seg_diagnostico'}).drop(['Codigo'], axis=1)
df = df.merge(cid_df, how='left', left_on='cid10_diagnostico_alta', right_on='Codigo').rename(columns={'Descricao': 'descricao_diagnostico_alta'}).drop(['Codigo'], axis=1)
df = df.merge(cid_df, how='left', left_on='cid10_seg_diagnostico_alta', right_on='Codigo').rename(columns={'Descricao': 'descricao_seg_diagnostico_alta'}).drop(['Codigo'], axis=1)

In [ ]:
# Tabela das 'familias' de códigos CID-10
cid_fam = pd.read_csv('https://raw.githubusercontent.com/ImagineDogs/TranstornosMentais/main/CID-10%20-%20Familias.csv')
cid_fam.head()

In [ ]:
# Inicializando as colunas a serem preenchidas pelas familias
df['cid10_familia'] = np.nan
df['cid10_descricao_familia'] = np.nan
df['cid10_seg_familia'] = np.nan
df['cid10_seg_descricao_familia'] = np.nan
df['cid10_familia_alta'] = np.nan
df['cid10_descricao_familia_alta'] = np.nan
df['cid10_seg_familia_alta'] = np.nan
df['cid10_seg_descricao_familia_alta'] = np.nan

In [ ]:
# Realizando as junções por query
for i in range(len(cid_fam)):
  cod = cid_fam['Codigos'].iloc[i].split('-')

  df['cid10_familia'].loc[df['cid10_diagnostico'].between(cod[0], cod[1])] = cid_fam['Codigos'].iloc[i]
  df['cid10_descricao_familia'].loc[df['cid10_diagnostico'].between(cod[0], cod[1])] = cid_fam['Descricao'].iloc[i]

  df['cid10_seg_familia'].loc[df['cid10_seg_diagnostico'].between(cod[0], cod[1])] = cid_fam['Codigos'].iloc[i]
  df['cid10_seg_descricao_familia'].loc[df['cid10_seg_diagnostico'].between(cod[0], cod[1])] = cid_fam['Descricao'].iloc[i]

  df['cid10_familia_alta'].loc[df['cid10_diagnostico_alta'].between(cod[0], cod[1])] = cid_fam['Codigos'].iloc[i]
  df['cid10_descricao_familia_alta'].loc[df['cid10_diagnostico_alta'].between(cod[0], cod[1])] = cid_fam['Descricao'].iloc[i]

  df['cid10_seg_familia_alta'].loc[df['cid10_seg_diagnostico_alta'].between(cod[0], cod[1])] = cid_fam['Codigos'].iloc[i]
  df['cid10_seg_descricao_familia_alta'].loc[df['cid10_seg_diagnostico_alta'].between(cod[0], cod[1])] = cid_fam['Descricao'].iloc[i]

In [ ]:
df.to_csv('df_cid10.csv')